In [13]:
import os
import json

def read_json(fn):
    with open(fn, 'r') as f:
        return json.load(f)
    
ROOT_DATASET_DIR = '/datasets/COCO'
IMAGES_DIR = os.path.join(ROOT_DATASET_DIR, 'images')
ANNOTATIONS_JSON_PATH = os.path.join(ROOT_DATASET_DIR, 'annotations/instances_val2017.json')
COUNT = 100

annotations = read_json(ANNOTATIONS_JSON_PATH)

In [14]:
import random
ids = random.sample([x['id'] for x in annotations['images']], COUNT)

annotations['images'] = [x for x in annotations['images'] if x['id'] in ids]   
annotations['annotations'] = [x for x in annotations['annotations'] if x['image_id'] in ids]

len(annotations['images'])

100

In [17]:
with open(f"temp_{COUNT}.json", 'w') as f:
    json.dump(annotations, f, indent=4)

### Arch checker

In [6]:
import torch
from aimet_torch.arch_checker.arch_checker import ArchChecker
from mmdet.apis import DetInferencer

CONFIG_PATH = 'rtmdet_tiny_8xb32-300e_coco.py'
WEIGHTS_PATH = 'rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth'

dummy_input = torch.rand(1, 3, 640, 640)
model = DetInferencer(model=CONFIG_PATH, weights=WEIGHTS_PATH)

type(model.model)

Loads checkpoint by local backend from path: rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std



/home/shayaan/miniconda3/envs/aimet2/lib/python3.10/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


mmdet.models.detectors.rtmdet.RTMDet

In [12]:
from aimet_torch.quantsim import QuantizationSimModel, QuantScheme
from aimet_torch.model_preparer import prepare_model

sim_model = prepare_model(model.model)
sim2 = prepare_model(sim_model)
sim3 = prepare_model(sim2)
type(sim_model), type(sim2)

2024-08-02 13:08:19,578 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stem.0.bn.module_batch_norm} 
2024-08-02 13:08:19,579 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stem.1.bn.module_batch_norm_1} 
2024-08-02 13:08:19,579 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stem.2.bn.module_batch_norm_2} 
2024-08-02 13:08:19,579 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage1.0.bn.module_batch_norm_3} 
2024-08-02 13:08:19,580 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage1.1.short_conv.bn.module_batch_norm_4} 
2024-08-02 13:08:19,580 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage1.1.main_conv.bn.module_batch_norm_5} 
2024-08-02 13:08:19,581 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage1.1.blocks.0.conv1.bn.

2024-08-02 13:08:19,588 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage3.1.blocks.0.module_add_2} 
2024-08-02 13:08:19,589 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage3.1.module_cat_2} 
2024-08-02 13:08:19,589 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage3.1.attention.module_mul_2} 
2024-08-02 13:08:19,589 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage3.1.final_conv.bn.module_batch_norm_23} 
2024-08-02 13:08:19,589 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage4.0.bn.module_batch_norm_24} 
2024-08-02 13:08:19,590 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage4.1.conv1.bn.module_batch_norm_25} 
2024-08-02 13:08:19,590 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage4.1.module_cat_3} 
2024-08

(torch.fx.graph_module.GraphModule.__new__.<locals>.GraphModuleImpl,
 torch.fx.graph_module.GraphModule.__new__.<locals>.GraphModuleImpl)

In [10]:
ArchChecker.check_model_arch(model.model, dummy_input, "./")

/home/shayaan/Desktop/aimet/my_mmdet/mmdet/models/dense_heads/anchor_head.py:115: UserWarning: DeprecationWarning: anchor_generator is deprecated, please use "prior_generator" instead
  warnings.warn('DeprecationWarning: anchor_generator is deprecated, '
/home/shayaan/Desktop/aimet/my_mmdet/mmdet/models/dense_heads/anchor_head.py:108: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '


KeyError: '1'